## 나주시 농업기상정보시스템
* 주소 : https://weather.naju.go.kr/

In [1]:
import pandas as pd
from numpy import nan
import time

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [3]:
# 특이 사항
# 2016년 2월 : 29일
# 나머지 2월 : 28일

# 1,3,5,7,8,10,12 : 31일
# 4,6,9,11 : 30일

# 2013/01/04 18시 없음

# 2014/06/13 부터 2개 열이 추가 됨.
# 날짜와 기온 15번째마다 반복. 그 전까진 13번째마다 반복.
# 조건 추가해서 기온 크롤링 함수 작성.

In [4]:
# webdriver로 페이지 접속
driver = webdriver.Chrome("chromedriver_94")
url = "https://weather.naju.go.kr/agri_meteo/agri_time.html"
driver.get(url)
driver.implicitly_wait(10)

# bs로 크롤링하기

## 1년간 시간에 따른 기온 크롤링 함수
* 특이사항
    * 2014년 6월 12일까지 td리스트에서 13번째마다 날짜, 기온 반복.
    * 2014년 6월 12일이후부턴 열이 2개 추가되어 15번째마다 반목.
    * 다른 조건으로 크롤링.

    * 2016년 2월은 29일까지 존재.

In [5]:
def year_temp_data(year):
       
    all_date_list = []
    all_tp_list = []
    
    # 2013년
    if year == 2013:
        # 31일까지 있는 달
        for m in [1,3,5,7,8,10,12]:
            for d in range(1,32):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                n = m+1
                e = d+1
                z = year+1
                # 각 일 마지막 시간
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                # 각 달 마지막 일
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                # 각 연도 마지막 일
                if f"{z}-01-01 00:00" in date_list:
                    a = date_list.index(f"{z}-01-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"    
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 30일까지 있는 달
        for m in [4,6,9,11]:
            for d in range(1,31):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                            
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 2월 ~28일
        for m in [2]:
            for d in range(1,29):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                        
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

    # 2014년
    if year == 2014:
        # 2014년 6월전, 31일까지 있는 달
        for m in [1,3,5]:
            for d in range(1,32):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                n = m+1
                e = d+1
                z = year+1
                # 각 일 마지막 시간
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                # 각 달 마지막 일
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                # 각 연도 마지막 일
                if f"{z}-01-01 00:00" in date_list:
                    a = date_list.index(f"{z}-01-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"    
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 2014년 6월 13일전, 30일까지 있는 달
        for m in [4]:
            for d in range(1,31):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                            
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 2월 ~28일
        for m in [2]:
            for d in range(1,29):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                        
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 2014년 6월
        for m in [6]:
            # 6월 12일까지 td리스트에서 13번째마다 날짜, 기온 반복
            for d in range(1,13):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                        
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

            # 6월 13일부터 30일까지 td리스트에서 15번째마다 날짜, 기온 반복
            for d in range(13,31):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/15)
                date_list = []
                for i in range(num):
                    if len(td[i*15].text) == 16:
                        date_list.append(td[i*15].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                n = m+1
                e = d+1
                z = year+1
                # 각 일 마지막 시간
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                # 각 달 마지막 일
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 6/13이후 31일까지있는 달
        for m in [7,8,10,12]:
            for d in range(1,32):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/15)
                date_list = []
                for i in range(num):
                    if len(td[i*15].text) == 16:
                        date_list.append(td[i*15].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                n = m+1
                e = d+1
                z = year+1
                # 각 일 마지막 시간
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                # 각 달 마지막 일
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                # 각 연도 마지막 일
                if f"{z}-01-01 00:00" in date_list:
                    a = date_list.index(f"{z}-01-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"    
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 6/13이후 30일까지있는 달
        for m in [9,11]:
            for d in range(1,31):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/15)
                date_list = []
                for i in range(num):
                    if len(td[i*15].text) == 16:
                        date_list.append(td[i*15].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                n = m+1
                e = d+1
                z = year+1
                # 각 일 마지막 시간
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                # 각 달 마지막 일
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"    
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)


    # 2015, 2017, 2018년
    if year in [2015, 2017, 2018]:
        # 31일까지 있는 달
        for m in [1,3,5,7,8,10,12]:
            for d in range(1,32):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/15)
                date_list = []
                for i in range(num):
                    if len(td[i*15].text) == 16:
                        date_list.append(td[i*15].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                n = m+1
                e = d+1
                z = year+1
                # 각 일 마지막 시간
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                # 각 달 마지막 일
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                # 각 연도 마지막 일
                if f"{z}-01-01 00:00" in date_list:
                    a = date_list.index(f"{z}-01-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"    
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 30일까지 있는 달
        for m in [4,6,9,11]:
            for d in range(1,31):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/15)
                date_list = []
                for i in range(num):
                    if len(td[i*15].text) == 16:
                        date_list.append(td[i*15].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                            
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 2월 ~28일
        for m in [2]:
            for d in range(1,29):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/15)
                date_list = []
                for i in range(num):
                    if len(td[i*15].text) == 16:
                        date_list.append(td[i*15].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                        
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

    # 2016년 2월 ~29일
    if year == 2016:
        # 31일까지 있는 달
        for m in [1,3,5,7,8,10,12]:
            for d in range(1,32):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/15)
                date_list = []
                for i in range(num):
                    if len(td[i*15].text) == 16:
                        date_list.append(td[i*15].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                n = m+1
                e = d+1
                z = year+1
                # 각 일 마지막 시간
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                # 각 달 마지막 일
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                # 각 연도 마지막 일
                if f"{z}-01-01 00:00" in date_list:
                    a = date_list.index(f"{z}-01-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"    
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 30일까지 있는 달
        for m in [4,6,9,11]:
            for d in range(1,31):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/15)
                date_list = []
                for i in range(num):
                    if len(td[i*15].text) == 16:
                        date_list.append(td[i*15].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                            
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 2월 ~29일
        for m in [2]:
            for d in range(1,30):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/15)
                date_list = []
                for i in range(num):
                    if len(td[i*15].text) == 16:
                        date_list.append(td[i*15].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 15 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                        
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)
        
    return all_date_list, all_tp_list

In [6]:
# 2013년 날짜, 기온 크롤링
# 1,3,5,7,8,10,12 : 7개월 * 31일 = 217 개
# 4,6,9,11 : 4개월 * 30일 = 120 개
# 2 : 28일
# 2013년 일수 : 365일 * 24시간 = 8760 개
# 함수 year_temp_data(year)
date2013, temp2013 = year_temp_data(2013)
print(len(date2013), len(temp2013))

8760 8760


In [7]:
dat2013 = {"년-월-일 시간":date2013, "기온":temp2013}
df2013 = pd.DataFrame(dat2013)
df2013.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   년-월-일 시간  8760 non-null   object
 1   기온        8659 non-null   object
dtypes: object(2)
memory usage: 137.0+ KB


In [14]:
df2013.to_csv("2013년_시간별_기온.csv")

In [17]:
# 2014년 날짜, 기온 크롤링
# 2 : 28일
# 2014년 일수 : 365일 * 24시간 = 8760 개
# 함수 year_temp_data(year)
date2014, temp2014 = year_temp_data(2014)
print(len(date2014), len(temp2014))

8760 8760


In [18]:
dat2014 = {"년-월-일 시간":date2014, "기온":temp2014}
df2014 = pd.DataFrame(dat2014)
df2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   년-월-일 시간  8760 non-null   object
 1   기온        8727 non-null   object
dtypes: object(2)
memory usage: 137.0+ KB


In [19]:
df2014.to_csv("2014년_시간별_기온.csv")

In [20]:
# 2015년 날짜, 기온 크롤링
# 2 : 28일
# 2015년 일수 : 365일 * 24시간 = 8760 개
# 함수 year_temp_data(year)
date2015, temp2015 = year_temp_data(2015)
print(len(date2015), len(temp2015))

8760 8760


In [21]:
dat2015 = {"년-월-일 시간":date2015, "기온":temp2015}
df2015 = pd.DataFrame(dat2015)
df2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   년-월-일 시간  8760 non-null   object
 1   기온        8755 non-null   object
dtypes: object(2)
memory usage: 137.0+ KB


In [22]:
df2015.to_csv("2015년_시간별_기온.csv")

In [31]:
# 2016년 날짜, 기온 크롤링
# 2 : 29일
# 2015년 일수 : 366일 * 24시간 = 8784 개
# 함수 year_temp_data(year)
date2016, temp2016 = year_temp_data(2016)
print(len(date2016), len(temp2016))

8784 8784


In [32]:
dat2016 = {"년-월-일 시간":date2016, "기온":temp2016}
df2016 = pd.DataFrame(dat2016)
df2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   년-월-일 시간  8784 non-null   object
 1   기온        8782 non-null   object
dtypes: object(2)
memory usage: 137.4+ KB


In [33]:
df2016.to_csv("2016년_시간별_기온.csv")

In [34]:
# 2017년 날짜, 기온 크롤링
# 2 : 28일
# 2017년 일수 : 365일 * 24시간 = 8760 개
# 함수 year_temp_data(year)
date2017, temp2017 = year_temp_data(2017)
print(len(date2017), len(temp2017))

8760 8760


In [35]:
dat2017 = {"년-월-일 시간":date2017, "기온":temp2017}
df2017 = pd.DataFrame(dat2017)
df2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   년-월-일 시간  8760 non-null   object
 1   기온        8754 non-null   object
dtypes: object(2)
memory usage: 137.0+ KB


In [36]:
df2017.to_csv("2017년_시간별_기온.csv")

In [37]:
# 2018년 날짜, 기온 크롤링
# 2 : 28일
# 2018년 일수 : 365일 * 24시간 = 8760 개
# 함수 year_temp_data(year)
date2018, temp2018 = year_temp_data(2018)
print(len(date2018), len(temp2018))

8760 8760


In [38]:
dat2018 = {"년-월-일 시간":date2018, "기온":temp2018}
df2018 = pd.DataFrame(dat2018)
df2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   년-월-일 시간  8760 non-null   object
 1   기온        8756 non-null   object
dtypes: object(2)
memory usage: 137.0+ KB


In [39]:
df2018.to_csv("2018년_시간별_기온.csv")

In [40]:
all_date = []
all_date.extend(date2013)
all_date.extend(date2014)
all_date.extend(date2015)
all_date.extend(date2016)
all_date.extend(date2017)
all_date.extend(date2018)

In [41]:
all_temp = []
all_temp.extend(temp2013)
all_temp.extend(temp2014)
all_temp.extend(temp2015)
all_temp.extend(temp2016)
all_temp.extend(temp2017)
all_temp.extend(temp2018)

In [42]:
print(len(all_date), len(all_temp))

52584 52584


In [43]:
all_dict = {"연-월-일 시간":all_date, "기온":all_temp}
df = pd.DataFrame(all_dict)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52584 entries, 0 to 52583
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   연-월-일 시간  52584 non-null  object
 1   기온        52433 non-null  object
dtypes: object(2)
memory usage: 821.8+ KB


In [45]:
df.to_csv('2013-2018년_시간별_기온.csv')